# Building the main pathfinding function
---

In [1]:
import numpy as np
import pandas as pd

In [2]:
import heapq as hq

In [3]:
# table of stop data (prefiltered to those in service between regular weekdays from 5pm - 6pm)
stops = pd.read_pickle('data/model/model_stops.pickle')

In [4]:
# walking speed factor
wsf = pd.read_pickle('data/model/f.pickle')

In [5]:
stops.head(3)

,stop_id,stop_code,stop_name,stop_lat,stop_lon
1,263,929,Davenport Rd at Bedford Rd,43.674448,-79.399659
2,264,940,Davenport Rd at Dupont St,43.675511,-79.401938
3,265,1871,Davisville Ave at Cleveland St,43.702088,-79.378112


In [6]:
stop_id_array = stops.stop_id.unique()

In [7]:
# search to stops df for the closest stop id given a set of geocoordinates
def find_closest_stop_id(input_lat, input_lon):
    stop_distance = stops.loc[:, ['stop_id', 'stop_lat', 'stop_lon']]
    stop_distance['distance'] = ( abs(input_lat - stop_distance['stop_lat'])**2 + abs(input_lon - stop_distance['stop_lon'])**2 )**(1/2)
    closest_stop_id = stop_distance.sort_values(by = 'distance').stop_id.iloc[0]
    return closest_stop_id

In [8]:
# currently set to home address
s_location = (43.760442381532236, -79.33181073515874)

In [9]:
# currently set to North York General Hospital near Leslie & Sheppard
t_location = (43.769136021058905, -79.3627326936882)

In [10]:
# s => source, t => target
s_stop_id = find_closest_stop_id(s_location[0], s_location[1])
t_stop_id = find_closest_stop_id(t_location[0], t_location[1])

s_stop_id, t_stop_id

(917, 2592)

In [11]:
# confirm the selection of s & t stops
stops[(stops.stop_id == s_stop_id) | (stops.stop_id == t_stop_id)]

,stop_id,stop_code,stop_name,stop_lat,stop_lon
436,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751
1835,2592,13688,North York General Hospital - Main Entrance,43.769710,-79.363485


In [12]:
# create an empty graph df for pathfinding model to build upon
graph = stops.copy()
graph = graph.set_index('stop_id')
graph = graph.drop(columns = ['stop_code', 'stop_name'])
graph['duration'] = np.nan
graph['parent'] = np.nan
graph['transit'] = False

In [13]:
graph.head(3)

,stop_lat,stop_lon,duration,parent,transit
stop_id,,,,,
263,43.674448,-79.399659,NaN,NaN,False
264,43.675511,-79.401938,NaN,NaN,False
265,43.702088,-79.378112,NaN,NaN,False


In [14]:
def initialize_graph(df, x, y):
    lat = df.loc[x, 'stop_lat']
    lon = df.loc[x, 'stop_lon']
    distance = ((abs(df.stop_lat - lat)**2 + abs(df.stop_lon - lon)**2)**(1/2))
    df.duration = distance / y
    df.parent = x
    df.loc[x, 'parent'] = 0
    return df[['duration', 'parent', 'transit']]

In [15]:
def path_finder(G, s, t, f):
    
    G = initialize_graph(G, s, f)
    
    # initialize priority queue
    Q = list(zip(G.duration.values, G.index.values))
    hq.heapify(Q)
    
    return G, Q

In [16]:
g, q = path_finder(graph, s_stop_id, t_stop_id, wsf)

In [112]:
g

,duration,parent,transit
stop_id,,,
263,7645.422768,917,False
264,7688.468479,917,False
265,5189.225445,917,False
266,18889.560790,917,False
267,18960.300810,917,False
...,...,...,...
24418,6978.914307,917,False
24419,6752.501210,917,False
24420,6792.320585,917,False


In [109]:
def initialize_priority_queue(graph):
    duration = graph.duration.values
    index = graph.index.values
    counter = len(graph)
    queue = list(list(l) for l in zip(duration, range(0, counter), index))
    stop_lookup = dict(zip(index, queue))
    hq.heapify(queue)
    return queue, stop_lookup, counter

In [76]:
def update_priority_queue(queue, stop_lookup, counter, stop, duration):
    if stop in stop_lookup:
        mark_stop_as_invalid(stop_lookup, stop)
    counter += 1
    stop_entry = [duration, counter, stop]
    stop_lookup[stop] = stop_entry
    hq.heappush(queue, stop_entry)
    return queue, stop_lookup, counter

In [77]:
def mark_stop_as_invalid(stop_lookup, stop):
    stop_entry = stop_lookup.pop(stop)
    stop_entry[-1] = 'invalid_entry'
    return stop_lookup

In [78]:
def extract_min_from_priority_queue(queue, stop_lookup):
    while queue:
        duration, count, stop = hq.heappop(queue)
        if stop != 'invalid_entry':
            del stop_lookup[stop]
            return queue, stop_lookup, stop
    else:
        return queue, stop_lookup, 0

In [126]:
pq = []
qd = {}
c = 0

In [142]:
pq, qd, c = initialize_priority_queue(g.head(3))

In [143]:
print(pq)
print(qd)
print(c)

[[5189.225444638779, 2, 265], [7688.468478773767, 1, 264], [7645.422768034198, 0, 263]]
{263: [7645.422768034198, 0, 263], 264: [7688.468478773767, 1, 264], 265: [5189.225444638779, 2, 265]}
3


In [144]:
pq, qd, c = update_priority_queue(pq, qd, c, 917, 0)
print(pq)
print(qd)
print(c)

[[0, 4, 917], [5189.225444638779, 2, 265], [7645.422768034198, 0, 263], [7688.468478773767, 1, 264]]
{263: [7645.422768034198, 0, 263], 264: [7688.468478773767, 1, 264], 265: [5189.225444638779, 2, 265], 917: [0, 4, 917]}
4


In [145]:
pq, qd, c = update_priority_queue(pq, qd, c, 5181, 33)
print(pq)
print(qd)
print(c)

[[0, 4, 917], [33, 5, 5181], [7645.422768034198, 0, 263], [7688.468478773767, 1, 264], [5189.225444638779, 2, 265]]
{263: [7645.422768034198, 0, 263], 264: [7688.468478773767, 1, 264], 265: [5189.225444638779, 2, 265], 917: [0, 4, 917], 5181: [33, 5, 5181]}
5


In [146]:
pq, qd, c = update_priority_queue(pq, qd, c, 264, 30)
print(pq)
print(qd)
print(c)

[[0, 4, 917], [33, 5, 5181], [30, 6, 264], [7688.468478773767, 1, 'invalid_entry'], [5189.225444638779, 2, 265], [7645.422768034198, 0, 263]]
{263: [7645.422768034198, 0, 263], 265: [5189.225444638779, 2, 265], 917: [0, 4, 917], 5181: [33, 5, 5181], 264: [30, 6, 264]}
6


In [147]:
pq, qd, c = update_priority_queue(pq, qd, c, 7089, 27)
print(pq)
print(qd)
print(c)

[[0, 4, 917], [33, 5, 5181], [27, 7, 7089], [7688.468478773767, 1, 'invalid_entry'], [5189.225444638779, 2, 265], [7645.422768034198, 0, 263], [30, 6, 264]]
{263: [7645.422768034198, 0, 263], 265: [5189.225444638779, 2, 265], 917: [0, 4, 917], 5181: [33, 5, 5181], 264: [30, 6, 264], 7089: [27, 7, 7089]}
7


In [148]:
pq, qd, x = extract_min_from_priority_queue(pq, qd)
print(x)
print(pq)
print(qd)
print(c)

917
[[27, 7, 7089], [33, 5, 5181], [30, 6, 264], [7688.468478773767, 1, 'invalid_entry'], [5189.225444638779, 2, 265], [7645.422768034198, 0, 263]]
{263: [7645.422768034198, 0, 263], 265: [5189.225444638779, 2, 265], 5181: [33, 5, 5181], 264: [30, 6, 264], 7089: [27, 7, 7089]}
7


In [149]:
pq, qd, c = update_priority_queue(pq, qd, c, 5181, 60)
print(pq)
print(qd)
print(c)

[[27, 7, 7089], [33, 5, 'invalid_entry'], [30, 6, 264], [7688.468478773767, 1, 'invalid_entry'], [5189.225444638779, 2, 265], [7645.422768034198, 0, 263], [60, 8, 5181]]
{263: [7645.422768034198, 0, 263], 265: [5189.225444638779, 2, 265], 264: [30, 6, 264], 7089: [27, 7, 7089], 5181: [60, 8, 5181]}
8


In [150]:
pq, qd, c = update_priority_queue(pq, qd, c, 5181, 25)
print(pq)
print(qd)
print(c)

[[25, 9, 5181], [27, 7, 7089], [30, 6, 264], [33, 5, 'invalid_entry'], [5189.225444638779, 2, 265], [7645.422768034198, 0, 263], [60, 8, 'invalid_entry'], [7688.468478773767, 1, 'invalid_entry']]
{263: [7645.422768034198, 0, 263], 265: [5189.225444638779, 2, 265], 264: [30, 6, 264], 7089: [27, 7, 7089], 5181: [25, 9, 5181]}
9


In [151]:
pq, qd, x = extract_min_from_priority_queue(pq, qd)
print(x)
print(pq)
print(qd)
print(c)

5181
[[27, 7, 7089], [33, 5, 'invalid_entry'], [30, 6, 264], [7688.468478773767, 1, 'invalid_entry'], [5189.225444638779, 2, 265], [7645.422768034198, 0, 263], [60, 8, 'invalid_entry']]
{263: [7645.422768034198, 0, 263], 265: [5189.225444638779, 2, 265], 264: [30, 6, 264], 7089: [27, 7, 7089]}
9


In [152]:
pq, qd, x = extract_min_from_priority_queue(pq, qd)
print(x)
print(pq)
print(qd)
print(c)

7089
[[30, 6, 264], [33, 5, 'invalid_entry'], [60, 8, 'invalid_entry'], [7688.468478773767, 1, 'invalid_entry'], [5189.225444638779, 2, 265], [7645.422768034198, 0, 263]]
{263: [7645.422768034198, 0, 263], 265: [5189.225444638779, 2, 265], 264: [30, 6, 264]}
9


In [153]:
pq, qd, x = extract_min_from_priority_queue(pq, qd)
print(x)
print(pq)
print(qd)
print(c)

264
[[33, 5, 'invalid_entry'], [5189.225444638779, 2, 265], [60, 8, 'invalid_entry'], [7688.468478773767, 1, 'invalid_entry'], [7645.422768034198, 0, 263]]
{263: [7645.422768034198, 0, 263], 265: [5189.225444638779, 2, 265]}
9


In [154]:
pq, qd, x = extract_min_from_priority_queue(pq, qd)
print(x)
print(pq)
print(qd)
print(c)

265
[[7645.422768034198, 0, 263], [7688.468478773767, 1, 'invalid_entry']]
{263: [7645.422768034198, 0, 263]}
9


In [155]:
pq, qd, x = extract_min_from_priority_queue(pq, qd)
print(x)
print(pq)
print(qd)
print(c)

263
[[7688.468478773767, 1, 'invalid_entry']]
{}
9


In [156]:
pq, qd, x = extract_min_from_priority_queue(pq, qd)
print(x)
print(pq)
print(qd)
print(c)

0
[]
{}
9
